Set up

In [11]:
from pathlib import Path
home = str(Path.home()) # all other paths are relative to this path. change to something else if this is not the case on your system

%load_ext autoreload
%autoreload 2

# make sure your run the cell above before running this
import Lab4_helper
import Lab5_helper

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This assignment should be relatively straightforward. I'm going to reuse the code I wrote in lab 4 to create a decision tree and then modify the code I wrote in lab 5 to select a random subset of the features during bagging.

## Dataset from Lab 4
Since the dataset from lab 5 doesn't have many features but lab 4 does, random forest will have more dramatic results, since the whole thing that's special about random forest is taking a random subset of the features. If we don't, it's just bagging

loading the data in...

In [12]:
import pandas as pd
import numpy as np

diabetes_df = pd.read_csv(
    f"./diabetes_indicators.csv"
)
diabetes_df.head()

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


The same preprocessing we used in lab4...

In [13]:
features = ['Sex','Age','Education','Income','Fruits','Veggies','Smoker', "HighChol", "BMI"]
dia_X = diabetes_df.loc[:,features][:1000]
dia_X = dia_X.dropna()
dia_X

,Sex,Age,Education,Income,Fruits,Veggies,Smoker,HighChol,BMI
0,0.0,9.0,4.0,3.0,0.0,1.0,1.0,1.0,40.0
1,0.0,7.0,6.0,1.0,0.0,0.0,1.0,0.0,25.0
2,0.0,9.0,4.0,8.0,1.0,0.0,0.0,1.0,28.0
3,0.0,11.0,3.0,6.0,1.0,1.0,0.0,0.0,27.0
4,0.0,11.0,5.0,4.0,1.0,1.0,0.0,1.0,24.0
...,...,...,...,...,...,...,...,...,...
995,0.0,2.0,6.0,8.0,1.0,0.0,0.0,0.0,31.0
996,0.0,10.0,5.0,8.0,0.0,1.0,0.0,0.0,21.0
997,1.0,7.0,4.0,1.0,0.0,0.0,0.0,1.0,31.0
998,0.0,5.0,4.0,8.0,1.0,1.0,0.0,0.0,37.0


In [14]:

dia_t = diabetes_df.loc[dia_X.index,'Diabetes_012']
dia_t

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
995    0.0
996    0.0
997    0.0
998    0.0
999    0.0
Name: Diabetes_012, Length: 1000, dtype: float64

## Code for Lab4

I use my decision tree from lab4 straight out of the box, no modifacation needed. I do want to compare how it'll do against the other methods, and show that it works, so I'll run it on the dataset.

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

default = 0
results = {}
dia_X_train, dia_X_test, dia_t_train, dia_t_test = train_test_split(dia_X, dia_t, test_size=0.3, random_state = 0)
c45_tree = Lab4_helper.make_tree2(dia_X_train, dia_t_train)
rules_c45 = Lab4_helper.generate_rules(c45_tree)
y_c45 = dia_X_test.apply(lambda x: Lab4_helper.make_prediction(rules_c45,x,default),axis=1)
c45_f1 = f1_score(dia_t_test, y_c45, average="weighted")
results["c45"] = c45_f1
results

{'c45': 0.6431422505307854}

# Creating Random Forest Algorithm

This is just the bagging code but we take a random subset of the features for every bag. After doing some reading, it looks like it's best to use $\sqrt{n}$ features for every learner for classifacation problems, where n is the number of features in total. 

We do have the modify the code to use our scratch random forest instead of ouf the scikit learn implementation

In [16]:
from sklearn.utils import resample

def make_tree_rf(X,y,min_split_count=5, prevFeature = "", prevPrevFeature = ""):
    tree = {}
    differentYVals = y.unique()
    defaultAns = y.value_counts().sort_values(ascending=False).index[0]

    #selecting a subset of the features
    numFeatures = int(np.round_(np.sqrt(X.shape[1])))
    X = X.sample(n = numFeatures, axis="columns")
    
    if(differentYVals.size) == 1:
        return defaultAns
    if(X.shape[0] < min_split_count):
        return defaultAns
    if(X.shape[1] == 0):
        return defaultAns
    if(X.drop_duplicates().shape[0] == 1):
        return defaultAns
    #then we're in the recursive case
    
    
    #pick the field with the highest IG
    bestFeature, rig = Lab4_helper.select_split2(X, y)
    if rig <= 0.001:
        return defaultAns
    
    #replacing the continous column with its split counterpart
    bestFeatureName = bestFeature.name
    originalColumnName = bestFeatureName.split("<")[0]
    X = X.drop(columns=[originalColumnName])
    X[bestFeatureName] = bestFeature

    #setting up tree
    tree[bestFeatureName] = {}
    possibleFeatureValues = [True, False]
    
    #recursing
    for value in possibleFeatureValues:
        XAtVal = X.loc[X[bestFeatureName] == value]
        YAtVal = y.loc[X[bestFeatureName] == value]     
        RestOfX = XAtVal.drop(columns=bestFeatureName)
        tree[bestFeatureName][str(value).capitalize()] = make_tree_rf(RestOfX, YAtVal, prevFeature=bestFeatureName, prevPrevFeature=prevFeature)
    
    return tree

def make_trees(x, t, ntrees):
    trees = []
    for _ in range(ntrees):
        #grabbing a random resample of the rows
        x_i, y_i = resample(x, t)
        tree = make_tree_rf(x_i, y_i)
        trees.append(tree)

    return trees

def make_rules(trees):
    rules = []
    for tree in trees:
        rule = Lab4_helper.generate_rules(tree)
        rules.append(rule)
    
    return rules

def make_pred_from_rules(x, rules, default = 0):
    predictions = pd.DataFrame()
    for rule in rules:
        #making the prediction from a single truee
        pred = x.apply(lambda x: Lab4_helper.make_prediction(rule,x,default),axis=1)
        #adding it to our dataframe of predictions
        predictions = pd.concat([predictions, pred], axis = 1)

    #we want the prediction to be the most populator prediction from our subset of predictions
    aggregate_prediction = predictions.mode(axis=1)
    return aggregate_prediction

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

def run_test(X, t, results, type, alg, ntrials, ntrees):
    scores = []
    for trial in range(ntrials):
        #splitting
        X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=0.3,random_state=trial)
        #performing algorithm
        y = None
        if alg == "random_forest" and type == "classifier":
            trees = make_trees(X_train, t_train, ntrees=ntrees)
            rules = make_rules(trees)
            y = make_pred_from_rules(X_test, rules, default=default)
        if alg == "bagging":
            trees = Lab5_helper.make_trees(X_train, t_train,ntrees=ntrees, type=type)
            y = Lab5_helper.make_prediction(trees, X_test, type=type)
        if alg == "sk_random_forest" and type == "classifier":
            model = RandomForestClassifier(n_estimators=ntrees, min_samples_split=5, random_state=trial, criterion="entropy").fit(X_train, t_train)
            y = model.predict(X_test)
        if alg == "boosting" and type == "classifier":
            boostingClassifier = GradientBoostingClassifier(n_estimators=ntrees).fit(X_train, t_train)
            y = boostingClassifier.predict(X_test)
        #calculating performance
        if type == "classifier":
            scores.append(f1_score(y, t_test, average="weighted"))
        elif type == "regressor":
            scores.append(np.sqrt(((y-t_test)**2).sum()/len(t_test)))
    average_score = sum(scores) / len(scores)
    results[alg] = average_score

In [18]:
classification_results = {}
run_test(X=dia_X, t= dia_t, results=classification_results, type="classifier", alg="random_forest", ntrials=10, ntrees=5)
classification_results

{'random_forest': 0.8569341877380964}

In [19]:
run_test(X=dia_X, t= dia_t, results=classification_results, type="classifier", alg="bagging", ntrials=10, ntrees=5)
classification_results

{'random_forest': 0.8569341877380964, 'bagging': 0.7374099015470598}

In [20]:
run_test(X=dia_X, t= dia_t, results=classification_results, type="classifier", alg="sk_random_forest", ntrials=10, ntrees=25)
classification_results

{'random_forest': 0.8569341877380964,
 'bagging': 0.7374099015470598,
 'sk_random_forest': 0.7977920649619576}

In [21]:
run_test(X=dia_X, t= dia_t, results=classification_results, type="classifier", alg="boosting", ntrials=10, ntrees=25)
classification_results

{'random_forest': 0.8569341877380964,
 'bagging': 0.7374099015470598,
 'sk_random_forest': 0.7977920649619576,
 'boosting': 0.819052444520835}

Comparing to built in implementation

Okay sweet, that's a noticable improvement from a single tree!
Lets compare it to bagging and boosting.
The following code will be pretty much straight lab 5 code

This is really cool! The random forest algorithm did better than than built-in boosting algorithm did!

# Applying the algorithms to other datasets

I'm going to use a dataset that contains the chemical properties of various red wines and their numerical rating of "quality". I'm going to try to do a regression between those properties and the quality. I obtained the dataset from https://archive.ics.uci.edu/ml/datasets/wine+quality

Loading the dataset in and preprocessing
displaying the raw dataset:

In [22]:
df = pd.read_csv("./winequality-red.csv", delimiter=";")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


displaying the preprocessed X:

In [23]:
X = df.drop(columns=["quality"])
t = df["quality"]
X = X.apply(lambda col: (col - col.mean()) / col.std())
X.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,-0.528194,0.961576,-1.391037,-0.453077,-0.243630,-0.466047,-0.379014,0.558100,1.288240,-0.579025,-0.959946
1,-0.298454,1.966827,-1.391037,0.043403,0.223805,0.872365,0.624168,0.028252,-0.719708,0.128910,-0.584594
2,-0.298454,1.296660,-1.185699,-0.169374,0.096323,-0.083643,0.228975,0.134222,-0.331073,-0.048074,-0.584594
3,1.654339,-1.384011,1.483689,-0.453077,-0.264878,0.107558,0.411372,0.664069,-0.978798,-0.461036,-0.584594
4,-0.528194,0.961576,-1.391037,-0.453077,-0.243630,-0.466047,-0.379014,0.558100,1.288240,-0.579025,-0.959946


Applying random forest

Because this is a regression problem, I'll be using 1/3 of the features for each tree.

In [24]:
wine_results = {}
ntrials = 10
ntrees = 50
RMSEs = []
default = 0
for trial in range(ntrials):
    X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=0.3,random_state=trial)
    trees = Lab5_helper.make_trees(X_train,t_train,ntrees=ntrees, random_forest=True)
    y = Lab5_helper.make_prediction(trees,X_test)
    RMSEs.append(np.sqrt(((y-t_test)**2).sum()/len(t_test)))
average_RMSE = sum(RMSEs) / len(RMSEs)
wine_results["random_forest"] = average_RMSE
wine_results

{'random_forest': 0.6089917169428827}

Applying bagging

In [25]:
RMSEs = []
for trial in range(ntrials):
    X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=0.3,random_state=trial)
    trees = Lab5_helper.make_trees(X_train,t_train,ntrees=ntrees)
    y = Lab5_helper.make_prediction(trees,X_test)
    RMSEs.append(np.sqrt(((y-t_test)**2).sum()/len(t_test)))
average_RMSE = sum(RMSEs) / len(RMSEs)
wine_results["bagging"] = average_RMSE
wine_results

{'random_forest': 0.6089917169428827, 'bagging': 0.5895205271926941}

Applying Boosting

In [26]:
RMSEs = []
for trial in range(ntrials):
    X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=0.3,random_state=trial)
    X_train2, X_val, t_train2, t_val = train_test_split(X_train, t_train, test_size=0.3,random_state=trial)
    trees,train_RMSEs,val_RMSEs = Lab5_helper.make_trees_boost(X_train2, X_val, t_train2, t_val, max_ntrees=100)
    trees = Lab5_helper.cut_trees(trees,val_RMSEs)
    y = Lab5_helper.make_prediction_boost(trees,X_test)
    RMSEs.append(np.sqrt(((y-t_test)**2).sum()/len(t_test)))
average_RMSE = sum(RMSEs) / len(RMSEs)
wine_results["boosting"] = average_RMSE
wine_results

{'random_forest': 0.6089917169428827,
 'bagging': 0.5895205271926941,
 'boosting': 0.6962058795294425}

Interestingly, random forest performed slightly worse than bagging alone, but both bagging methods perfomed better than boosting

# Feature Importance

## Gini Gain of a Tree

## Gini Gain at a Node

### Gini Impurity
The Gini Impurity score at node $n$ is defined as the probability of picking two different classes if you picked randomly from all the samples at node n. It ranges from 0 to 1 and the higher it is, the more impure the data and the lower it is, the more pure the data. It's mathematically described as$$g(n) = 1- \sum_{i=1}^{j}(P_i)^2$$
where:

$n$ is the current node (split in the dataset based on the value of some feature)

$j$ is the number of distinct classes,

$P_i$ is what portion the $i$-th class is of all the classes (the probaiblity of selecting the $i$-th class at random)


We can implement it with this function

In [27]:
def gini(x):
    counts = x.value_counts()
    fracs = counts / len(x)
    ans = 1 - (fracs ** 2).sum()
    return ans

Testing with a "impure" dataset A and a "pure" dataset B

In [28]:
A = pd.Series([1,2, 2, 3, 4, 4, 1, 5, 5])
B = pd.Series([0, 1, 1, 1, 1, 0, 1, 1, 0])

print("Impurity  of A is", gini(A))
print("Impurity of B is", gini(B))

Impurity  of A is 0.7901234567901234
Impurity of B is 0.4444444444444444


Looks good!

### Mean Gini Impurity Gain
We're interested in average decrease in gini impurity at the current node -- aka average increase in purity. That is, the difference between the gini impurity at $n$ and the weighted sum of the gini impurity of its two children. The weight of each child is what proportion of the samples at $n$ are included in the child. Mathematically this is
$$gg(n) = g(n) - \sum_{i = 1}^{c} \frac{s_i}{s_n}g(c_i)$$

Where:

$n$ is the current node

$gg(n)$ is the mean gini impurity decrease at $n$

$c$ is the number of children

$s_i$ is the number of samples included in the $i$-th child

$s_n$ is the number of samples at $n$

$g(c_i)$ is the gini impurity of the $i$-th child node

we can implement it with this function

In [29]:
def split_data(x, t, tree):
    feature_name, threshold = list(tree.keys())[0].split("<")
    threshold = float(threshold)

    #Split the data
    x_l = x[x[feature_name] < threshold]
    x_r = x[x[feature_name] >= threshold]
    t_l = t[x[feature_name] < threshold]
    t_r = t[x[feature_name] >= threshold]

    return x_l, x_r, t_l, t_r

def gid(x, t, tree):
    #split the data by the metric in the tree. The node n is the head node of the tree. 
    #Grab the metric in question
    x_l, x_r, t_l, t_r  = split_data(x, t, tree)
    
    #calculate gid
    p_l = len(x_l) / len(x)
    p_r = len(x_r) / len(x)

    gini_n = gini(t)
    gini_l = gini(t_l)
    gini_r = gini(t_r)

    ans = gini_n - (p_l * gini_l + p_r * gini_r)
    return ans

Let's test the function on a table that can be neatly divided in two

In [30]:
data = {"isHot": [0, 0, 0, 1, 1, 1], "shouldTouch": [1, 1, 1, 0, 0, 0,]}
test_df = pd.DataFrame(data)
tree = {
    "isHot<0.5": {"False": 1,
                  "True": 0}
}
test_x = test_df.drop(columns=["shouldTouch"])
test_t = test_df.drop(columns=["isHot"])
gid(test_x, test_t, tree)

0.5

This makes sense! As the impurity at the start is 0.5, and the purity of each of the split tables is 0, so the weighted decrease in impurity is 0.5

Testing on Lab 4 Code

In [31]:

gid(dia_X_test, dia_t_test, c45_tree)

0.015036706311076897

Does that seem reasonable? I have no idea!

### Feature Importance from Gini Impurity Decrease
For every tree, for every node, we find the Gini impurity decrease and then weight it by the proportion of the  number of samples at the node to the number of samples in total. We take that value and average it over each feature. This is our feature importance metric.

We can implement this like so:

In [32]:
#function for recursing through tree and calculating gid at every node:
def gini_importance_from_tree(x, t, tree, n_samples, feature_results):
    if len(x) == 0:
        return 
    if not isinstance(tree, dict):
        return
    feature_name, threshold = list(tree.keys())[0].split("<")
    gid_i  = gid(x, t, tree)
    importance = gid_i * (len(x) / n_samples)
    if feature_name in feature_results:
        feature_results[feature_name].append(importance)
    else:
        feature_results[feature_name] = list([importance])

    #recursing
    subtree = list(tree.values())[0]
    for expected_value, next_tree in subtree.items():
        sub_x = x[(x[feature_name] < float(threshold)) == (expected_value == "True")]
        sub_t = t[(x[feature_name] < float(threshold)) == (expected_value == "True")]
        gini_importance_from_tree(sub_x, sub_t, next_tree, n_samples, feature_results)
    

In [33]:
ntrials = 10
ntrees = 25
default = 0
feature_results = {}
for trial in range(ntrials):
    X_train, X_test, t_train, t_test = train_test_split(dia_X, dia_t, test_size=0.3,random_state=trial)
    trees = make_trees(X_train,t_train,ntrees=ntrees)
    for tree in trees:
        gini_importance_from_tree(dia_X_train, dia_t_train, tree, len(dia_X_train), feature_results)

data = {key: sum(value) / len(value) for key, value in feature_results.items()}
pd.Series(data).sort_values(ascending=False)


Age          0.006983
BMI          0.006818
Income       0.003870
HighChol     0.002311
Veggies      0.001816
Education    0.000982
Sex          0.000378
Fruits       0.000311
Smoker       0.000305
dtype: float64

This makes sense! This metrics intuitively seem important for estimating BMI

In [28]:
from sklearn.ensemble import RandomForestClassifier
ntrials = 10
ntrees = 100
default = 0
sk_feature_results = {}
for trial in range(ntrials):
    X_train, X_test, t_train, t_test = train_test_split(dia_X, dia_t, test_size=0.3,random_state=trial)
    classifier = RandomForestClassifier(n_estimators=ntrees, random_state=trial, min_samples_split=5, criterion="entropy").fit(X_train,t_train)
    for i in range(len(classifier.feature_names_in_)):
        feature = classifier.feature_names_in_[i]
        importance = classifier.feature_importances_[i]
        if feature not in sk_feature_results:
            sk_feature_results[feature] = [importance]
        else:
            sk_feature_results[feature].append(importance)

pd.DataFrame(sk_feature_results).T.mean(axis=1).sort_values(ascending=False)

BMI          0.277710
Age          0.210713
Income       0.165448
Education    0.110505
Fruits       0.052029
HighChol     0.046748
Sex          0.045992
Smoker       0.045799
Veggies      0.045056
dtype: float64

What's more is that it has very similar results to the built-in implementation! THe top 4 are all the same and all of the other values are only off by very small amounts. 